# Deep Learning with keras

In this workshop, we will try to build some feedforward models to do sentiment analysis, using keras, a deep learning library: https://keras.io/

You will need pandas, keras (2.3.1) and tensorflow (2.1.0; and their dependencies) to run this code (pip install pandas keras==2.3.1 tensorflow-cpu==2.1.0).

First let's prepare the data. We are using 1000 yelp reviews, nnotated with either positive or negative sentiments.

In [1]:
import pandas as pd

corpus = "07-yelp-dataset.txt"
df = pd.read_csv(corpus, names=['sentence', 'label'], sep='\t')
print("Number of sentences =", len(df))
print("\nData:")
print(df.iloc[:3])

Number of sentences = 1000

Data:
                                    sentence  label
0                   Wow... Loved this place.      1
1                         Crust is not good.      0
2  Not tasty and the texture was just nasty.      0


Next, let's create the train/dev/test partitions

In [2]:
import random
import numpy as np

sentences = df['sentence'].values
labels = df['label'].values

#partition data into 80/10/10 for train/dev/test
sentences_train, y_train = sentences[:800], labels[:800]
sentences_dev, y_dev = sentences[800:900], labels[800:900]
sentences_test, y_test = sentences[900:1000], labels[900:1000]

#convert label list into arrays
y_train = np.array(y_train)
y_dev = np.array(y_dev)
y_test = np.array(y_test)

print(y_train[0], sentences_train[0])
print(y_dev[0], sentences_dev[0])
print(y_test[0], sentences_test[0])

1 Wow... Loved this place.
0 I'm super pissd.
0 Spend your money elsewhere.


Let's tokenize the text. In this workshop, we'll use the ``tokenizer`` function provided by keras. Once the data is tokenized, we can then use ``texts_to_matrix`` to get the bag-of-words representation for each document.

In [3]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token="<UNK>")
tokenizer.fit_on_texts(sentences_train)

x_train = tokenizer.texts_to_matrix(sentences_train, mode="count") #BOW representation
x_dev = tokenizer.texts_to_matrix(sentences_dev, mode="count") #BOW representation
x_test = tokenizer.texts_to_matrix(sentences_test, mode="count") #BOW representation

vocab_size = x_train.shape[1]
print("Vocab size =", vocab_size)
print(x_train[0])

Using TensorFlow backend.


Vocab size = 1811
[0. 0. 0. ... 0. 0. 0.]


Before we build a neural network model, let's see how well logistic regression do with this dataset.

In [4]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
score = classifier.score(x_test, y_test)

print("Accuracy:", score)

Accuracy: 0.69


/Users/jiachengye/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


The logistic regression result is not too bad, and it will serve as a baseline for the deep learning models.

Now let's build a very simple feedforward network. Here the input layer is the BOW features, and we have one hidden layer (dimension = 10) and an output layer in the model.

In [5]:
from keras.models import Sequential
from keras import layers

#model definition
model = Sequential(name="feedforward-bow-input")
model.add(layers.Dense(10, input_dim=vocab_size, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

#since it's a binary classification problem, we use a binary cross entropy loss here
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "feedforward-bow-input"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                18120     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 18,131
Trainable params: 18,131
Non-trainable params: 0
_________________________________________________________________


Let's train the model. Notice that there are a few hyper-parameters here, e.g. hidden layer size, number of epochs and batch_size, and in practice these hyper-parameters should be tuned according to the development data to get an optimal model. In this workshop we'll use 20 epochs and a batch size of 10 (no further tuning). Once the model is trained, we'll compute the test accuracy performance.

In [7]:
#training
model.fit(x_train, y_train, epochs=20, verbose=True, validation_data=(x_dev, y_dev), batch_size=10)

loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
predicts = model.predict(x_test)
print("\nTesting Accuracy:  {:.4f}".format(accuracy))
print(predicts)

Train on 800 samples, validate on 100 samples
Epoch 1/20
800/800 [==============================] - 0s 93us/step - loss: 0.0240 - accuracy: 1.0000 - val_loss: 0.4053 - val_accuracy: 0.8400
Epoch 2/20
800/800 [==============================] - 0s 87us/step - loss: 0.0216 - accuracy: 1.0000 - val_loss: 0.4080 - val_accuracy: 0.8500
Epoch 3/20
800/800 [==============================] - 0s 85us/step - loss: 0.0195 - accuracy: 1.0000 - val_loss: 0.4147 - val_accuracy: 0.8500
Epoch 4/20
800/800 [==============================] - 0s 84us/step - loss: 0.0177 - accuracy: 1.0000 - val_loss: 0.4148 - val_accuracy: 0.8700
Epoch 5/20
800/800 [==============================] - 0s 88us/step - loss: 0.0160 - accuracy: 1.0000 - val_loss: 0.4257 - val_accuracy: 0.8500
Epoch 6/20
800/800 [==============================] - 0s 85us/step - loss: 0.0145 - accuracy: 1.0000 - val_loss: 0.4274 - val_accuracy: 0.8900
Epoch 7/20
800/800 [==============================] - 0s 77us/step - loss: 0.0133 - accuracy: 1.

How does the performance compare to logistic regression? If you run it a few times you may find that it gives slightly different numbers, and that is due to random initialisation of the model parameters.

Even though we did not explicitly define any word embeddings in the model architecture, they are in our model: in the weights between the input and the hidden layer. The hidden layer can therefore be interpreted as a sum of word embeddings for each input document.

Let's fetch the word embeddings of some words, and look at their cosine similarity, and see if they make any sense.

In [9]:
from numpy import dot
from numpy.linalg import norm

embeddings = model.get_layer(index=0).get_weights()[0] #word embeddings layer

emb_love = embeddings[tokenizer.word_index["love"]] #embeddings for 'love'
emb_like = embeddings[tokenizer.word_index["like"]]
emb_lukewarm = embeddings[tokenizer.word_index["lukewarm"]]
emb_bad = embeddings[tokenizer.word_index["bad"]]

print(emb_love)

def cos_sim(a, b):
    return dot(a, b)/(norm(a)*norm(b))

print("love vs. like =", cos_sim(emb_love, emb_like))
print("love vs. lukewarm =", cos_sim(emb_love, emb_lukewarm))
print("love vs. bad =", cos_sim(emb_love, emb_bad))
print("lukewarm vs. bad =", cos_sim(emb_lukewarm, emb_bad))

[ 0.27030575  0.34660634  0.18217364 -0.22256967  0.2348451   0.2069386
 -0.20672679 -0.2902843  -0.24853283  0.23060597]
love vs. like = 0.7652442
love vs. lukewarm = -0.9745116
love vs. bad = -0.9607358
lukewarm vs. bad = 0.9527822


Not bad. You should find that for *love* and *like*, which are both positive sentiment words, produce high cosine similarity. Similar observations for *lukewarm* and *bad*. But when we compare opposite polarity words like *love* and *bad*, we get negative cosine similarity values.

Next, we are going to build another feed-forward model, but this time, instead of using BOW features as input, we want to use the word sequence as input (so order of words is preserved). It is usually not straightforward to do this for classical machine learning models, but with neural networks and embeddings, it's pretty straightforward.

Let's first tokenise the input documents into word sequences.

In [10]:
#tokenise the input into word sequences

xseq_train = tokenizer.texts_to_sequences(sentences_train)
xseq_dev = tokenizer.texts_to_sequences(sentences_dev)
xseq_test = tokenizer.texts_to_sequences(sentences_test)

print(xseq_train[0])

[354, 138, 9, 17]


Because documents have variable lengths, we need to first 'pad' them to make all documents have the same length. keras uses word index 0 to represent 'pad symbols'.

In [8]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 2000
xseq_train = pad_sequences(xseq_train, padding='post', maxlen=maxlen)
xseq_dev = pad_sequences(xseq_dev, padding='post', maxlen=maxlen)
xseq_test = pad_sequences(xseq_test, padding='post', maxlen=maxlen)
print(xseq_train[0])

NameError: name 'xseq_train' is not defined

Now let's build our second model. This model first embeds each word in the input sequence into embeddings, and then concatenate the word embeddings together to represent input sequence. The ``Flatten`` function you see after the embedding layer is essentially doing the concatenation, by 'chaining' the list of word embeddings into a very long vector.

If our word embeddings has a dimension 10, and our documents always have 30 words (padded), then here the concatenated word embeddings have a dimension of 10 x 30 = 300. 

The concatenated word embeddings undergo a linear transformation with non-linear activations (``layers.Dense(10, activation='relu')``), producing a hidden representation with a dimension of 10. It is then passed to the output layer.

In [12]:
embedding_dim = 10

#word order preserved with this architecture
model2 = Sequential(name="feedforward-sequence-input")
model2.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model2.add(layers.Flatten())
model2.add(layers.Dense(10, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model2.summary()

Model: "feedforward-sequence-input"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 10)            18110     
_________________________________________________________________
flatten_1 (Flatten)          (None, 300)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                3010      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 21,131
Trainable params: 21,131
Non-trainable params: 0
_________________________________________________________________


Now let's train the model and compute the test accuracy.

In [13]:
model2.fit(xseq_train, y_train, epochs=20, verbose=True, validation_data=(xseq_dev, y_dev), batch_size=10)

loss, accuracy = model2.evaluate(xseq_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

/Users/jiachengye/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 800 samples, validate on 100 samples
Epoch 1/20
800/800 [==============================] - 0s 269us/step - loss: 0.6836 - accuracy: 0.5725 - val_loss: 0.7190 - val_accuracy: 0.4400
Epoch 2/20
800/800 [==============================] - 0s 94us/step - loss: 0.6631 - accuracy: 0.5987 - val_loss: 0.7144 - val_accuracy: 0.4400
Epoch 3/20
800/800 [==============================] - 0s 85us/step - loss: 0.6001 - accuracy: 0.7188 - val_loss: 0.6785 - val_accuracy: 0.5400
Epoch 4/20
800/800 [==============================] - 0s 84us/step - loss: 0.4370 - accuracy: 0.8975 - val_loss: 0.6378 - val_accuracy: 0.6400
Epoch 5/20
800/800 [==============================] - 0s 81us/step - loss: 0.2440 - accuracy: 0.9675 - val_loss: 0.5974 - val_accuracy: 0.6600
Epoch 6/20
800/800 [==============================] - 0s 86us/step - loss: 0.1311 - accuracy: 0.9812 - val_loss: 0.5614 - val_accuracy: 0.7200
Epoch 7/20
800/800 [==============================] - 0s 83us/step - loss: 0.0770 - accuracy: 0

You may find that the performance isn't as good as the BOW model. In general, concatenating word embeddings isn't a good way to represent word sequence.

A better way is to build a recurrent model. But first, let's extract the word embeddings for the 4 words as before and look at their similarity.

In [14]:
embeddings = model2.get_layer(index=0).get_weights()[0] #word embeddings

emb_love = embeddings[tokenizer.word_index["love"]]
emb_like = embeddings[tokenizer.word_index["like"]]
emb_lukewarm = embeddings[tokenizer.word_index["lukewarm"]]
emb_bad = embeddings[tokenizer.word_index["bad"]]

print("love vs. like =", cos_sim(emb_love, emb_like))
print("love vs. lukewarm =", cos_sim(emb_love, emb_lukewarm))
print("love vs. bad =", cos_sim(emb_love, emb_bad))
print("lukewarm vs. bad =", cos_sim(emb_lukewarm, emb_bad))

love vs. like = 0.1426105
love vs. lukewarm = -0.26727965
love vs. bad = -0.4823963
lukewarm vs. bad = 0.18090342


Now, let's try to build anç. After the embeddings layer, the LSTM layer will process the words one at a time, and compute the next state (dimension for the hidden state = 10 in this case). The output of the LSTM layer is the final state, produced after processing the last word, and that will be fed to the output layer.

In [25]:
x = [0.00,
0.00,
0.00,
0.00,
0.00,
0.00,
0.00,
0.00,
20.49,
23.83,
23.07,
22.18,
21.59,
24.62,
16.78,
16.03,
14.57,
13.55,
13.81,
21.35,
18.32,
20.41,
18.68,
21.33,
19.61,
23.03,
19.38,
22.26,
18.90,
19.26,
23.55,
21.46,
20.54,
18.98,
20.93,
20.96,
18.55,
21.46,
21.46,
16.69,
15.41,
16.45,
18.13,
21.78,
18.51,
22.79,
21.57,
30.28,
27.70,
28.71,
26.13,
19.72,
23.84,
21.45,
23.22,
28.90,
29.14,
21.46,
24.44,
22.04,
22.28,
25.29,
25.12,
19.74,
21.46,
20.61,
19.39,
22.76,
17.85,
17.53,
18.39,
21.28,
20.75,
18.48,
16.95,
25.75,
19.69,
24.97,
24.93,
30.84,
22.60,
25.37,
26.15,
23.64]

from numpy import array
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
X, y = split_sequence(x, 3)

from keras.models import Sequential
from keras import layers
from keras.layers import LSTM, Dense

n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(3, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X, y, epochs=200, verbose=0)

p = model.predict(X)
numacc = 0
for i in range(len(p)):
    if abs(p[i]-y[i]) < 5:
        numacc += 1
print(numacc/len(p))
x_input = array([x[81], x[82], x[83]])
x_input = x_input.reshape((1, 3, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

0.8888888888888888
[[23.444084]]


In [15]:
from keras.layers import LSTM

#word order preserved with this architecture
model3 = Sequential(name="lstm")
model3.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model3.add(LSTM(10))
model3.add(layers.Dense(1, activation='sigmoid'))
model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model3.summary()

Model: "lstm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 10)            18110     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 18,961
Trainable params: 18,961
Non-trainable params: 0
_________________________________________________________________


Let's train the LSTM model and see the test performance.

In [16]:
model3.fit(xseq_train, y_train, epochs=20, verbose=True, validation_data=(xseq_dev, y_dev), batch_size=10)

loss, accuracy = model3.evaluate(xseq_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

/Users/jiachengye/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 800 samples, validate on 100 samples
Epoch 1/20
800/800 [==============================] - 1s 2ms/step - loss: 0.6874 - accuracy: 0.5650 - val_loss: 0.7127 - val_accuracy: 0.4400
Epoch 2/20
800/800 [==============================] - 1s 972us/step - loss: 0.6835 - accuracy: 0.5650 - val_loss: 0.7106 - val_accuracy: 0.4400
Epoch 3/20
800/800 [==============================] - 1s 932us/step - loss: 0.6067 - accuracy: 0.6662 - val_loss: 0.4907 - val_accuracy: 0.8100
Epoch 4/20
800/800 [==============================] - 1s 922us/step - loss: 0.3115 - accuracy: 0.9038 - val_loss: 0.4887 - val_accuracy: 0.8100
Epoch 5/20
800/800 [==============================] - 1s 903us/step - loss: 0.1719 - accuracy: 0.9575 - val_loss: 0.5009 - val_accuracy: 0.8000
Epoch 6/20
800/800 [==============================] - 1s 898us/step - loss: 0.1175 - accuracy: 0.9737 - val_loss: 0.6441 - val_accuracy: 0.8200
Epoch 7/20
800/800 [==============================] - 1s 1ms/step - loss: 0.1027 - accuracy:

You should notice that the training is quite a bit slower, and that's because now the model has to process the sequence one word at a time. But the results should be better!

And lastly, let's extract the embeddings and look at the their similarity.

In [17]:
embeddings = model3.get_layer(index=0).get_weights()[0] #word embeddings

emb_love = embeddings[tokenizer.word_index["love"]]
emb_like = embeddings[tokenizer.word_index["like"]]
emb_lukewarm = embeddings[tokenizer.word_index["lukewarm"]]
emb_bad = embeddings[tokenizer.word_index["bad"]]

print("love vs. like =", cos_sim(emb_love, emb_like))
print("love vs. lukewarm =", cos_sim(emb_love, emb_lukewarm))
print("love vs. bad =", cos_sim(emb_love, emb_bad))
print("lukewarm vs. bad =", cos_sim(emb_lukewarm, emb_bad))

love vs. like = 0.467547
love vs. lukewarm = -0.8988657
love vs. bad = -0.8930899
lukewarm vs. bad = 0.9082503
